This file is used to take the url's from the initial collection of routes and scrape the "stats" web page for each route with the ultimate goal of collecting the ID of each climber who has climbed that route.

In [70]:
from pymongo import MongoClient
import json, re, requests, pprint, urllib, time
from bs4 import BeautifulSoup
from mp_api import MP
import numpy as np
import pandas as pd
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [31]:
key_path = '../data/mp_api_key.txt'
with open(key_path, 'r') as key_text:
    key = key_text.read()

In [32]:
url_path = '../data/route_urls.txt'
with open(url_path, 'r') as url_text:
    route_urls = url_text.read().splitlines()

In [72]:
client = MongoClient('localhost', 27017)
db = 'mountain_project'
collection = 'route_stats_page'
if collection in client[db].list_collection_names():
    (client[db])[collection].drop()

region_db_creation = MP(key, client, db, collection)

for route in route_urls:
    region_db_creation.scrape_url(route)
    time.sleep(1)

In [75]:
soup = BeautifulSoup(region_db_creation.collection.find_one()['route_stats_page'], 'html')
print('route_id', 'user_id')
print(region_db_creation.collection.find_one()['route_id'],
      soup.find('table', {'class': "table table-striped"}).find_all('a')[0]['href'].split('/')[4])

route_id user_id
105806397 10333


In [74]:
region_db_creation.collection.count_documents({})

4982

In [84]:
unique_user_ids = []
unique_route_ids = []

for counter, route_page in enumerate(region_db_creation.collection.find()):
    unique_route_ids.append(route_page['route_id'])
    soup = BeautifulSoup(route_page['route_stats_page'], 'html')
    try:
        for user in soup.find('table', {'class': "table table-striped"}).find_all('a'):
            unique_user_ids.append(user['href'].split('/')[4])
    except:
        print(f'No data provided for {route_page["route_id"]}, counter: {counter}')

unique_user_ids = list(set(unique_user_ids))

No data provided for 106935343, counter: 1433
No data provided for 106813219, counter: 2231
No data provided for 109326535, counter: 3036
No data provided for 108850918, counter: 3459
No data provided for 117849202, counter: 4024
No data provided for 117849159, counter: 4025
No data provided for 105797867, counter: 4308
No data provided for 111075283, counter: 4431
No data provided for 117783580, counter: 4442
No data provided for 114425782, counter: 4461
No data provided for 114425729, counter: 4483
No data provided for 114425763, counter: 4503
No data provided for 114425776, counter: 4505
No data provided for 114425748, counter: 4507
No data provided for 114425737, counter: 4512
No data provided for 117783393, counter: 4536
No data provided for 114425723, counter: 4574
No data provided for 112447859, counter: 4798


In [85]:
with open('../data/route_ids.txt', 'w') as out:
    [out.write(route_id + '\n') for route_id in unique_route_ids]
    
with open('../data/user_ids.txt', 'w') as out:
    [out.write(user_id + '\n') for user_id in unique_user_ids]